In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../data/movies_metadata.csv',low_memory=False).head(20000)

df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
#filtrar somente as features necessárias 
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [4]:
#Converter release_date em formato datetime 
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

#Extrair ano da data
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [5]:
#Substitui valores nulos NaT em 0 e anos em inteiros
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

#coluna ano
df['year'] = df['year'].apply(convert_int)

#Drop coluna não necessária
df = df.drop('release_date', axis=1)
df.head(3)

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995


In [6]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [7]:
from ast import literal_eval

#Verifica e testa o que função faz
a = "[1,2,3]"
print(type(a))

b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [8]:
#Converte todos NaN em listas vazias
df['genres'] = df['genres'].fillna('[]')

#Applica literal_eval
df['genres'] = df['genres'].apply(literal_eval)

#Converte lista de dicionarios em lista de strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

df.head(3)

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995


In [9]:
#Cria nova feature usando explode nos generos
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Nomeia nova feature
s.name = 'genre'

#Cria novo datataframe removendo coluna genres
gen_df = df.drop('genres', axis=1).join(s)

gen_df.head(3)

<ipython-input-9-430177615ffe>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family


In [10]:
def build_chart(gen_df, percentile=0.8):
    print("Entre com gênero preferido")
    genre = input()
    
    print("Entre com a menor duração")
    low_time = int(input())
    
    print("Entre com a maior duração")
    high_time = int(input())
    
    print("Entre com o menor ano")
    low_year = int(input())
    
    print("Entre com o maior ano")
    high_year = int(input())
    
    #Define nova variável para armazenar filmes 
    movies = gen_df.copy()
    
    #Filtragem baseado no que usuário escolheu
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Computa os valores de C e m para os filmes filtrados
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Considera filmes com pelo menos m votos.
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculascore usando a fórmula IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Ordena
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [11]:
#Gera catálago baseado nas escolhas e métrica
build_chart(gen_df).head()

Entre com gênero preferido
animation
Entre com a menor duração
30
Entre com a maior duração
120
Entre com o menor ano
1990
Entre com o maior ano
2005


,title,runtime,vote_average,vote_count,year,genre,score
359,The Lion King,89.0,8.0,5520.0,1994,animation,7.838855
9698,Howl's Moving Castle,119.0,8.2,2049.0,2004,animation,7.783312
0,Toy Story,81.0,7.7,5415.0,1995,animation,7.567913
6232,Finding Nemo,100.0,7.6,6292.0,2003,animation,7.493891
4756,"Monsters, Inc.",92.0,7.5,6150.0,2001,animation,7.401141


In [12]:
#Salva dataframe limpo em csv
df.to_csv('../../data/metadata_clean1.csv', index=False)